# GoNB ECharts Examples

They are adaptations of examples in [Apache EChart](https://echarts.apache.org/v4/examples/en/index.html) and in [go-echarts library](https://github.com/go-echarts/examples)

The first cell below redirects the `gonb-echarts` and `gonb` libraries to the local disk repository -- this was used during development but feel free to comment it out.

In [1]:
!*go mod edit --replace github.com/janpfeifer/gonb-echarts=${HOME}/Projects/gonb-echarts
!*go mod edit --replace github.com/janpfeifer/gonb=${HOME}/Projects/gonb

## Bar Chart

In [2]:
import (
	"math/rand"
	"github.com/go-echarts/go-echarts/v2/charts"
	"github.com/go-echarts/go-echarts/v2/opts"
    gonb_echarts "github.com/janpfeifer/gonb-echarts"
    "github.com/janpfeifer/must"   
)

// generate random data for bar chart
func generateBarItems() []opts.BarData {
	items := make([]opts.BarData, 0)
	for i := 0; i < 7; i++ {
		items = append(items, opts.BarData{Value: rand.Intn(300)})
	}
	return items
}

%%
bar := charts.NewBar()
// set some global options like Title/Legend/ToolTip or anything else
bar.SetGlobalOptions(charts.WithTitleOpts(opts.Title{
    Title:    "My first bar chart generated by go-echarts",
    Subtitle: "It's extremely easy to use, right?",
}))

// Put data into instance
bar.SetXAxis([]string{"Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"}).
    AddSeries("Category A", generateBarItems()).
    AddSeries("Category B", generateBarItems())

// Display
must.M(gonb_echarts.Display(bar, "width: 1024px; height:400px; background: white;"))

## Stacked Lines

In [3]:
func toLineData[In any](data []In) []opts.LineData {
    r := make([]opts.LineData, len(data))
    for ii, v := range data {
        r[ii].Value = v 
    }
    return r
}

%%
stackedLine := charts.NewLine()
stackedLine.SetGlobalOptions(
    charts.WithTitleOpts(opts.Title{Title: "Stacked Line",}), 
    charts.WithTooltipOpts(opts.Tooltip{Show:true, Trigger: "axis"}),
)
seriesOpt := charts.WithLineChartOpts(opts.LineChart{
    Stack: "Total",
    ShowSymbol: true,
})

stackedLine.
    SetGlobalOptions(charts.WithYAxisOpts(opts.YAxis{Type: "value"}))
stackedLine.
    SetXAxis([]string{"Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"}). 
    AddSeries("Email", toLineData([]int{120, 132, 101, 134, 90, 230, 210}), seriesOpt).
    AddSeries("Union Ads", toLineData([]int{220, 182, 191, 234, 290, 330, 310}), seriesOpt).
    AddSeries("Video Ads", toLineData([]int{150, 232, 201, 154, 190, 330, 410}), seriesOpt).
    AddSeries("Direct", toLineData([]int{320, 332, 301, 334, 390, 330, 320}), seriesOpt).
    AddSeries("Search Engine", toLineData([]int{820, 932, 901, 934, 1290, 1330, 1320}), seriesOpt)

must.M(gonb_echarts.Display(stackedLine, "width: 1024px; height:400px; background: white;"))

## Polar Example

Not working yet, see issue [go-echarts #414](https://github.com/go-echarts/go-echarts/issues/414)

In [4]:
import "math"

%%
const numPoints = 100
data := make([]opts.LineData, numPoints)
for ii := range 100 {
    theta := (float64(ii) / numPoints) * 360.0;
    r := 5 * (1.0 + math.Sin((theta / 180) * math.Pi));
    data[ii] = opts.LineData{Value: []float64{r, theta}}
}

polar := charts.NewLine()
polar.SetGlobalOptions(
    charts.WithTitleOpts(opts.Title{Title: "Two Value-Axes in Polar",}), 
    charts.WithPolarOps(opts.Polar{
        Tooltip: opts.Tooltip{
            Show: true,
            Trigger: "axis",
            AxisPointer: &opts.AxisPointer{Type: "cross"},
        },
    }),
    charts.WithAngleAxisOps(opts.AngleAxis{
        PolarAxisBase: opts.PolarAxisBase{
            Type: "value",
            StartAngle: 0.0,
        },
    }),
    charts.WithRadiusAxisOps(opts.RadiusAxis{
        PolarAxisBase: opts.PolarAxisBase{
            Type: "value",
            PolarIndex: 1,
        },
    }),
)
polar.AddSeries(
    "line", data, charts.WithCoordinateSystem("polar"))
// must.M(gonb_echarts.Display(polar, "width: 1024px; height:400px; background: white;"))

## Geo Example

In [5]:
import (
	"io"
	"math/rand"
	"os"

    "github.com/janpfeifer/gonb/gonbui"
    gonb_echarts "github.com/janpfeifer/gonb-echarts"
    "github.com/janpfeifer/must"   
	"github.com/go-echarts/go-echarts/v2/charts"
	"github.com/go-echarts/go-echarts/v2/components"
	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/types"
)

var geoData = []opts.GeoData{
	{Name: "北京", Value: []float64{116.40, 39.90, float64(rand.Intn(100))}},
	{Name: "上海", Value: []float64{121.47, 31.23, float64(rand.Intn(100))}},
	{Name: "重庆", Value: []float64{106.55, 29.56, float64(rand.Intn(100))}},
	{Name: "武汉", Value: []float64{114.31, 30.52, float64(rand.Intn(100))}},
	{Name: "台湾", Value: []float64{121.30, 25.03, float64(rand.Intn(100))}},
	{Name: "香港", Value: []float64{114.17, 22.28, float64(rand.Intn(100))}},
}

var guangdongData = []opts.GeoData{
	{Name: "汕头", Value: []float64{116.69, 23.39, float64(rand.Intn(100))}},
	{Name: "深圳", Value: []float64{114.07, 22.62, float64(rand.Intn(100))}},
	{Name: "广州", Value: []float64{113.23, 23.16, float64(rand.Intn(100))}},
}

func geoBase() *charts.Geo {
	geo := charts.NewGeo()
	geo.SetGlobalOptions(
		charts.WithTitleOpts(opts.Title{Title: "basic geo example"}),
		charts.WithGeoComponentOpts(opts.GeoComponent{
			Map:       "china",
			ItemStyle: &opts.ItemStyle{Color: "#006666"},
		}),
	)

	geo.AddSeries("geo", types.ChartEffectScatter, geoData,
		charts.WithRippleEffectOpts(opts.RippleEffect{
			Period:    4,
			Scale:     6,
			BrushType: "stroke",
		}),
	)
	return geo
}

func geoGuangdong() *charts.Geo {
	geo := charts.NewGeo()
	geo.SetGlobalOptions(
		charts.WithTitleOpts(opts.Title{Title: "Guangdong province"}),
		charts.WithGeoComponentOpts(opts.GeoComponent{
			Map: "广东",
		}),
		charts.WithVisualMapOpts(opts.VisualMap{
			Calculable: true,
			InRange: &opts.VisualMapInRange{
				Color: []string{"#50a3ba", "#eac736", "#d94e5d"},
			},
		}),
	)

	geo.AddSeries("geo", types.ChartScatter, guangdongData)
	return geo
}

type GeoExamples struct{}

%%
content := []string{
    must.M1(gonb_echarts.DisplayContent(geoBase(), "width: 100%; height:400px; background: white;")),
    must.M1(gonb_echarts.DisplayContent(geoGuangdong(), "width: 100%; height:400px; background: white;")),
}
gonbui.DisplayHtmlf(`<div style="display: flex">%s%s</div>`, content[0], content[1])